In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import data, segmentation, feature, future
from sklearn.ensemble import RandomForestClassifier
from functools import partial

full_img = data.skin()

img = full_img[:900, :900]
img.shape

In [ ]:
from src.dataset import Dataset

import torch.utils.data
import matplotlib.pyplot as plt
file_name_zueri = r'D:\Temp\AgroLuege\raw_data\ZueriCrop\ZueriCrop.hdf5'
traindataset= Dataset(file_name_zueri, 0., 'train', False, 1, "labels.csv",time_downsample_factor=1, num_channel=3, apply_cloud_masking=False,small_train_set_mode=False)


In [ ]:
traindataloader = torch.utils.data.DataLoader(traindataset, batch_size=11190, shuffle=True, num_workers=0)

In [ ]:
for iteration, data in enumerate(traindataloader):
    input, target, target_2, target_3 = data
    print(data)
    data_i = data

In [ ]:
input= input[0:6000]
target_3 = target_3[0:6000]

In [ ]:
reshaped_tensor = input.reshape(24*60, 24*100, 3)
reshaped_target = target_3.reshape(24*60, 24*100)

In [ ]:
sigma_min = 1
sigma_max = 16
features_func = partial(feature.multiscale_basic_features,
                        intensity=True, edges=False, texture=True,
                        sigma_min=sigma_min, sigma_max=sigma_max,
                        channel_axis=-1)
features = features_func(reshaped_tensor)


In [ ]:
clf = RandomForestClassifier(n_estimators=50, n_jobs=-1,
                             max_depth=10, max_samples=0.05)
clf = future.fit_segmenter(reshaped_target, features, clf)
result = future.predict_segmenter(features, clf)


In [ ]:
fig, ax = plt.subplots(1, 2, sharex=True, sharey=True, figsize=(9, 4))
ax[0].imshow(segmentation.mark_boundaries(reshaped_tensor.numpy(), result, mode='thick'))
# ax[0].contour(reshaped_target.numpy())
ax[0].set_title('Image, mask and segmentation boundaries')
ax[1].imshow(result)
ax[1].set_title('Segmentation')
fig.tight_layout()

In [ ]:
np.sum(result == reshaped_target.numpy())/len(reshaped_target.flatten())*100

In [ ]:
unique, counts = np.unique(result, return_counts=True)

In [ ]:
print(np.asarray((unique, counts)).T)